In [7]:
from consensus.PBFT_Simulator import PBFT_Simulator
from consensus.QPBFT_Simulator import QPBFT_Simulator
from consensus.Proposed_Simulator import Proposed_Simulator
import time

In [8]:
consensus = PBFT_Simulator(10, 2)

In [9]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
delay = end-start
print(f"Spend time => {delay} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 1, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 2, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 3, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 4, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 5, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 6, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 7, Faulty => False, Send_messages => ('Hello world 0', 'pre-prepare', 7), Receive_messages => [('Hello world 0', 'pre-prepare', 7, '0d705f4356a9632ec71bad238a7ab1b1a47f36b7025a9d942f80005b78b4baa0')]
IdUser => 8, Faulty => True

In [10]:
del consensus

In [11]:
consensus = QPBFT_Simulator(6, 4, 2)

In [12]:
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
spend = end-start
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Role =>  Manager, Faulty =>  False, Send_message_log => ('Hello world 0', 'prepare', 0)    , Receive_messages_log => [('Hello world 0', 'prepare', 0, '5310eca6643a80e0bafcc136e6c89b61921827e21289714b66752fb1d9425448')]
IdUser => 1, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 2, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 3, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 4, Role =>  Manager, Faulty =>   True, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 5, Role =>  Manager, Faulty =>  False, Send_message_log => ''                                 , Receive_messages_log => []
IdUser => 6, Role =>    Voter, Faulty =>  False, Send_message_log => ''                  

In [13]:
del consensus

In [14]:
consensus = Proposed_Simulator(6, 4, 5, 3)
start = time.time()
for i in range(10):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} seconds")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 1, Role =>  Master, send_message_log => ''                                 , messages_log => []
IdUser => 6, Role =>   Slave, send_message_log => ''                                 , messages_log => []
IdUser => 7, Role =>   Slave, send_message_log => ''                                 , messages_log => [('Hello world 0', 'request', -1)]
IdUser => 5, Role =>  Master, send_message_log => ''                                 , messages_log => []
IdUser => 9, Role =>   Slave, send_message_log => ''                                 , messages_log => []
------------------------------
Prepare Internal Phase
IdUser => 1, Role =>  Master, send_message_log => ''                                 , messages_log => [('Hello world 0', 'prepare', 7)]
IdUser => 6, Role =>   Slave, send_message_log => ''                                 , messages_log => [('Hello world 0', 'prepare', 7)]
IdUser => 7, Role =>   Slave, send_message_log => ('Hello world 0', 'prepare', 7)    , messages_

In [15]:
del consensus